In [1]:
import string

In [2]:
#checks if two generators are inverses of each other, generators entered as lists
def is_inverse(generator1,generator2): 
    if generator2[0]==generator1[0]+"^-1" or generator1[0]==generator2[0]+"^-1":
        if generator1[1]==generator2[1]:
            return True
    return False
    

In [3]:
print(is_inverse(["a",3],["a",3]))
print(is_inverse(["a",3],["a",2]))
print(is_inverse(["a^-1",3],["a",3]))
print(is_inverse(["a^-1","3"],["a",3])) # **is index int or str?**
print(is_inverse(["a^-1",2],["a",3]))

False
False
True
False
False


In [4]:
#relation and generator are inputed already adjusted eg. [["a",""],["b^-1",""],["c",""],["a",""]] 
#computes exponent sum of a generator
def exponent_sum(generator,relation):
    positive_exponent=0
    negative_exponent=0
    for letter in relation:
        if letter==generator: 
            positive_exponent+=1
        elif is_inverse(letter,generator)==True:
            negative_exponent+= 1
    return positive_exponent-negative_exponent

In [5]:
#we take in relation as string and converts it into a list of lists. This is only used at the beginning.
def relation_adjustment(relation):
    #relation=list(relation)
    relation_list=[]
    for character in relation:
        if character.isalpha():
            relation_list.append(character)
        else:
            relation_list[len(relation_list)-1]=relation_list[len(relation_list)-1]+character
    for i in range(len(relation_list)):
        relation_list[i]=[relation_list[i],""]
    return relation_list

In [6]:
print(relation_adjustment("ab^-1c"))
print(relation_adjustment("a^2b^-1c"))

[['a', ''], ['b^-1', ''], ['c', '']]
[['a^2', ''], ['b^-1', ''], ['c', '']]


In [7]:
print(exponent_sum(["c",""],relation_adjustment("a^-1bcaab^-1ccac^-1")))
print(exponent_sum(["a",""],relation_adjustment("a^-1bcaab^-1ccac^-1aaa")))

2
5


In [8]:
#Takes generators as a list, and returns list of lists where the second element in each list
#is it's index. If it has no index, there will be empty string. eg a_-1 is [a,-1], b^-1 is [b^-1,""]
def generators_adjustment(generators):
    generators_list=[]
    for generator in generators:
        generators_list.append([generator,""])
    return generators_list

In [9]:
print(generators_adjustment(["a","b","c"]))
print(generators_adjustment(['a_-100',"a_-1"])) # **unfinished**

[['a', ''], ['b', ''], ['c', '']]
[['a_-100', ''], ['a_-1', '']]


In [10]:
#returns the inverse of an element (entered as a list)
def inverse(element):
    if len(element[0])==1:
        element[0]=element[0]+"^-1"
        return element
    else:
        element[0]=element[0][0]
        return element

In [11]:
print(inverse(["a","3"]))
print(inverse(['a^-1','99']))

['a^-1', '3']
['a', '99']


In [12]:
#Here we rewrite relation so that the generator with exponent sum=0 is not in the first position
#relation and generators are already adjusted
def remove_exp_zero(zero_generator,relation):
    new_relation=relation
    n=0
    for letter in relation:
        if letter==zero_generator or letter==inverse(zero_generator):
            new_relation=new_relation[1:]
            new_relation.append(inverse(letter))
        else:
            return new_relation
    return new_relation

In [13]:
print(remove_exp_zero(["a","3"],[["a^-1","3"],["b",""],["c",3]])) # **type of index is inconsistent, str or int?**
print(remove_exp_zero(["a",3],[["a",3],["a^-1",3],["b",""],["c",3]]))
print(remove_exp_zero(["a",3],[["b",""],["c",3]]))
print(remove_exp_zero(["a",""],relation_adjustment("ab^-1ac")))

[['b', ''], ['c', 3], ['a', '3']]
[['b', ''], ['c', 3], ['a^-1', 3], ['a', 3]]
[['b', ''], ['c', 3]]
[['b^-1', ''], ['a', ''], ['c', ''], ['a^-1', '']]


In [14]:
#rewrites adjusted relation and puts subscripts
#it takes in the relation after having removed the generator with exponent sum 0 from the start(or its inverse)
def relation_prime(zero_generator,relation):
    new_relation=[]
    for i in relation:
        if zero_generator not in i:
            new_relation.append(i+str(-1*exp_counter(zero_generator,i,sylls)))
    return new_relation

In [15]:
inverse(["a",""])

['a^-1', '']

In [16]:
relation_prime(['a',''], relation_adjustment('bacbba^-1a^-1a^-1cccaa'))

NameError: name 'exp_counter' is not defined

In [ ]:
#input is zero_generator and relation already adjusted
#we make a list with each element in the relation and the exp sum of the zero generator before the element
def exp_counter(zero_generator,relation):
    exp_list=[]
    for letter in relation:
        alpha=0
        if letter != zero_generator:
            for j in relation:
                if j==zero_generator:
                    alpha=alpha+1
                elif j==inverse(zero_generator):
                    alpha=alpha-1
                elif j==letter:
                    exp_list.append([letter,alpha])
                    letter[0]=letter[0]+" flag"
                    print(relation)
                    break
    return exp_list


In [ ]:
print(relation_adjustment("b^-1aaca^-1a^-1cd"))
exp_counter(["a",""],relation_adjustment("b^-1aaca^-1a^-1cd"))

In [ ]:
relation_adjustment("b^-1aaca^-1a^-1cd")

In [ ]:
#Returns the subscript of a syllable
def syllable_subscript(syllable):
    if "-" in syllable[len(syllable)-2]:
        return int(syllable[len(syllable)-2:])
    else:
        return int(syllable[len(syllable)-1])

In [ ]:
#for a generator, we compute the smallest subscript appearing on that generator in relation_prime
#we do not imput zero_generator for generator here
def smallest_subscript(generator,relation_prime):
    for syllable in relation_prime:
        if generator in syllable:
            smallest_subscript=syllable_subscript(syllable)
#here we set smallest_subscript to the first subscript that appears in a syllable with the generator
    for syllable in relation_prime:
        if generator in syllable and syllable_subscript(syllable)<smallest_subscript:
            smallest_subscript=syllable_subscript(syllable)
    return smallest_subscript

In [ ]:
def largest_subscript(generator,relation_prime):
    for syllable in relation_prime:
        if generator in syllable:
            largest_subscript=syllable_subscript(syllable)
#here we set smallest_subscript to the first subscript that appears in a syllable with the generator
    for syllable in relation_prime:
        if generator in syllable and syllable_subscript(syllable)>largest_subscript:
            largest_subscript=syllable_subscript(syllable)
    return largest_subscript

In [ ]:
#Returns a list of generators from relation_prime
def generator_retriever(relation_prime):
    generator_list=[]
    for syllable in relation_prime:
        generator_of_syllable=syllable[0]
        if generator_of_syllable not in generator_list:
            generator_list.append(generator_of_syllable)
    return generator_list
        

In [ ]:
#Returns a relation in a list back as a string
def relation_string(relation_prime):
    relation_str=""
    for element in relation_prime:
        relation_str=relation_str+element
    return relation_str

In [ ]:
#Returns the presentation of intermediate group H. G is isomorphic to an HNN extension of H.
def subgroup_pres(relation_prime):
    generator_list=generator_retriever(relation_prime)
    subgroup_generators=[]
    for generator in generator_list:
        l_subscript=largest_subscript(generator,relation_prime)
        s_subscript=smallest_subscript(generator,relation_prime)
        indeces=list(range(s_subscript,l_subscript+1))
        for i in indeces:
            subgroup_generators.append(generator+f"_{i}")
    return [subgroup_generators, relation_string(relation_prime)]

In [ ]:
#Magnus Breakdown-one iteration
#input generators as a list, relation as a string
def magnus_breakdown(generators,relation):
    for generator in generators:
        if exponent_sum(generator,relation_adjustment(relation))==0:
            zero_generator=generator
            relation_temp=remove_exp_zero(zero_generator,relation_adjustment(relation))
            relation_temp=relation_prime(zero_generator,relation_temp)
        
    return subgroup_pres(relation_temp)

In [1]:
############################################################
############################################################
############################################################
############################################################
############################################################

In [2]:
class Syllable:
    def __init__(self, letter, subscript, exponent):
        self.ltr = letter    # string
        self.sub = subscript # int or empty string
        self.exp = exponent  # int
        
    def __str__(self):
        string = self.ltr
        if self.sub != "":
            string = string + "_{" + str(self.sub) + "}"
        if self.exp != 1:
            string = string + "^{" + str(self.exp) + "}"
        return string
    
    def inverse(self):
        new_exp = -self.exp
        return Syllable(self.ltr, self.sub, new_exp)
    
    def add_subscript(self, new_subscript): # Moves old subscript into letter
        if self.sub == "":
            return Syllable(self.ltr, new_subscript, self.exp)
        else:
            new_ltr = self.ltr + "_{" + str(self.sub) + "}"
            return Syllable(new_ltr, new_subscript, self.exp)
        
    def no_exp(self):
        return Syllable(self.ltr, self.sub, 1)

In [3]:
print(Syllable("a","",2))
s = Syllable("a",-3,2)
print("s        = " + str(s))
print("s^-1     = " + str(s.inverse()))
print("s is still " + str(s))
s_5 = s.add_subscript(5)
print("s_5      = " + str(s_5))
print("s is still " + str(s))

a^{2}
s        = a_{-3}^{2}
s^-1     = a_{-3}^{-2}
s is still a_{-3}^{2}
s_5      = a_{-3}_{5}^{2}
s is still a_{-3}^{2}


In [4]:
def str_to_syllable(string):
    letter = string[0]
    
    if "_" not in string: # e.g., a^5
        subscript = ""
    else:
        rest = string[string.index("_")+1:]
        if "^" in rest: # e.g., a_5^3
            subscript_str = rest[:rest.index("^")]
        else: # e.g., a^3_5
            subscript_str = rest
        subscript = int(subscript_str.replace("{","").replace("}",""))
    
    if "^" not in string: # e.g., b_2
        exponent = 1
    else:
        rest = string[string.index("^")+1:]
        if "_" in rest: # e.g., b^7_2
            exponent_str = rest[:rest.index("_")]
        else: # e.g., b_2^7
            exponent_str = rest
        exponent = int(exponent_str.replace("{","").replace("}",""))
    
    return Syllable(letter, subscript, exponent)

In [5]:
print(str_to_syllable("a_100^5"))
print(str_to_syllable("a^{5}_100"))
print(str_to_syllable("a^5"))
print(str_to_syllable("a_{100}"))
print(str_to_syllable("a"))

a_{100}^{5}
a_{100}^{5}
a^{5}
a_{100}
a


In [6]:
# A "word" is a list of syllables.
def str_to_word(string):
    string = string.replace(" ","")
    syllables = []
    syl_str = ""
    
    for i in range(len(string)):
        if not string[i].isalpha():
            syl_str = syl_str + string[i]
        else:
            if syl_str != "":
                syllables.append(str_to_syllable(syl_str))
            syl_str = string[i]
            
    syllables.append(str_to_syllable(syl_str))
    return(syllables)

def word_to_str(word):
    return " ".join([str(syllable) for syllable in word])

In [7]:
print(word_to_str(str_to_word("abc")))
print(word_to_str(str_to_word("a^2 b_100c^-1")))
print(word_to_str(str_to_word("a")))
print(word_to_str(str_to_word("aa")))

a b c
a^{2} b_{100} c^{-1}
a
a a


In [8]:
def reduce_word(syllables): # e.g., abaaa^-5 -> aba^-3
    i = 0
    while i < len(syllables)-1:
        s1 = syllables[i]
        s2 = syllables[i+1]
        if s1.ltr == s2.ltr and s1.sub == s2.sub:
            s1.exp = s1.exp + s2.exp
            syllables[i] = s1
            syllables.remove(s2)
        else:
            i = i+1
    return(syllables)

def invert_word(syllables): # e.g., a^2bc -> c^-1 b^-1 a^-2
    return [syl.inverse() for syl in list(reversed(syllables))]

In [9]:
w = str_to_word("aa a^2 a^-1 ab_1 b_1b_2")
w = reduce_word(w)
print(word_to_str(w))
w_inverse = invert_word(w)
print(word_to_str(w_inverse))

a^{4} b_{1}^{2} b_{2}
b_{2}^{-1} b_{1}^{-2} a^{-4}


In [10]:
def exponent_sumNEW(generator,relation):
    return sum([syl.exp for syl in relation
                if generator.ltr == syl.ltr and generator.sub == syl.sub])

In [11]:
print(exponent_sumNEW(str_to_syllable("a"),  str_to_word("a^2 b_100 a c^-1 a_0 a^-20")))
print(exponent_sumNEW(str_to_syllable("a_0"),str_to_word("a^2 b_100 a c^-1 a_0 a^-20 a^2_0")))

-17
3


In [12]:
#Here we rewrite relation so that the generator with exponent sum=0 is not in the first position
#relation and generators are already adjusted
def remove_exp_zeroNEW(zero_generator,relation):
    syl = relation[0]
    if zero_generator.ltr == syl.ltr and zero_generator.sub == syl.sub:
        return relation[1:] + [syl.inverse()]
    else:
        return relation

In [13]:
print(word_to_str(remove_exp_zeroNEW(str_to_syllable("a_3"), str_to_word("a^-1_3 b c_3"))))
print(word_to_str(remove_exp_zeroNEW(str_to_syllable("a_3"), str_to_word("b c_3"))))
print(word_to_str(remove_exp_zeroNEW(str_to_syllable("a_3"), str_to_word("a_3 b a_3 c_3"))))

b c_{3} a_{3}
b c_{3}
b a_{3} c_{3} a_{3}^{-1}


In [14]:
def relation_primeNEW(zero_generator,relation):
    new_relation = [relation[0].add_subscript(0)]
    i = 1
    while i < len(relation):
        if relation[i].ltr != zero_generator.ltr:
            subscript = -exponent_sumNEW(zero_generator, relation[:i])
            syl = relation[i].add_subscript(subscript)
            new_relation.append(syl)
        i = i+1
    return(new_relation)

In [15]:
r = str_to_word("b a c b^2 a^{-3} c^3 a^2")
print(word_to_str(r))
r_prime = relation_primeNEW(Syllable("a","",1), r)
print(word_to_str(r_prime))

b a c b^{2} a^{-3} c^{3} a^{2}
b_{0} c_{-1} b_{-1}^{2} c_{2}^{3}


In [16]:
def smallest_subscriptNEW(letter,relation):
    return min([syl.sub for syl in relation if syl.ltr == letter])

def largest_subscriptNEW(letter,relation):
    return max([syl.sub for syl in relation if syl.ltr == letter])

In [17]:
print(smallest_subscriptNEW("b", r_prime))
print(smallest_subscriptNEW("c", r_prime))

w = [Syllable("a_0",-100,1), Syllable("b",-7,9), Syllable("a_0",-20,99)]
print("w = " + word_to_str(w))
print(smallest_subscriptNEW("a_0", w))

print(largest_subscriptNEW("b", r_prime))
print(largest_subscriptNEW("c", r_prime))
print(largest_subscriptNEW("a_0", w))

-1
-1
w = a_0_{-100} b_{-7}^{9} a_0_{-20}^{99}
-100
0
2
-20


In [18]:
class Group:
    def __init__(self, generators, relations):
        self.gens = generators # list of syllables
        self.rels = relations  # list of words
        
    def __str__(self):
        gens_str = ", ".join([str(g) for g in self.gens])
        rels_str = ", ".join([word_to_str(r) for r in self.rels])
        return "<" + gens_str + " | "  + rels_str + ">"

In [19]:
def str_to_group(string):
    string = string.replace("<","").replace(">","").replace("\\langle","").replace("\\rangle","").replace(" ","")
    
    divider = "\\mid" if "\\mid" in string else "|"
    
    generators_string = string[:string.index(divider)]
    generators = [str_to_syllable(s) for s in generators_string.split(",")]

    relations_string = string[string.index(divider)+len(divider):]
    strings = relations_string.split(",")
    relations = []
    for s in strings:
        if "=" not in s: # e.g., xyx^{-1}y^{-1}
            relations.append(str_to_word(s))
        else: # e.g., xy=yx or xyx^-1=1
            left = s[:s.index("=")]
            left_word = str_to_word(left)
            
            right = s[s.index("=")+1:]
            right_word = str_to_word(right)
            
            if left == "1":
                relations.append(right_word)
            elif right == "1":
                relations.append(left_word)
            else:
                left_word.extend(invert_word(right_word))
                relations.append(left_word)
    
    return Group(generators, relations)

In [20]:
print(str_to_group("<a, b, c | b a c b^{2} a^{-3} c^{3} a^{2}>"))
print(str_to_group("\\langle a,b,c,q \\mid bac b^{2}   a^{-3} c^{3} a^{2},a^100 \\rangle"))
print(str_to_group("<x,y,a,b|xy=yx, a^2, b^100=1, 1=xyx^-1>"))

<a, b, c | b a c b^{2} a^{-3} c^{3} a^{2}>
<a, b, c, q | b a c b^{2} a^{-3} c^{3} a^{2}, a^{100}>
<x, y, a, b | x y x^{-1} y^{-1}, a^{2}, b^{100}, x y x^{-1}>


In [21]:
#Magnus Breakdown - one iteration
def magnus_one_step(group):
    generators = group.gens
    relation = group.rels[0]
    
    # Find a generator with exponent sum 0.
    for generator in generators:
        if exponent_sumNEW(generator, relation)==0:
            zero_generator = generator
            break
    else:
        # magnus_case2
        pass
    
    return magnus_case1(zero_generator, relation)

def magnus_case1(zero_generator, relation):
    relation_shifted = remove_exp_zeroNEW(zero_generator, relation)
    relation_prime = relation_primeNEW(zero_generator, relation_shifted)

    letters = {syl.ltr for syl in relation_prime}
    new_generators = []
    for letter in letters:
        l_subscript = largest_subscriptNEW(letter,relation_prime)
        s_subscript = smallest_subscriptNEW(letter,relation_prime)
        for i in range(s_subscript, l_subscript+1):
            new_generators.append(Syllable(letter, i, 1))
            
    return Group(new_generators, [relation_prime])

In [22]:
G = str_to_group("<a, b, c | b a c b^{2} a^{-3} c^{3} a^{2}>")
print(G)
H = magnus_one_step(G)
print(H)

<a, b, c | b a c b^{2} a^{-3} c^{3} a^{2}>
<b_{-1}, b_{0}, c_{-1}, c_{0}, c_{1}, c_{2} | b_{0} c_{-1} b_{-1}^{2} c_{2}^{3}>


In [23]:
G = str_to_group("<a,b|ab^10 ab^{-7} ab^{-3}>")
print(G)
H = magnus_one_step(G)
print(H)

<a, b | a b^{10} a b^{-7} a b^{-3}>
<a_{-10}, a_{-9}, a_{-8}, a_{-7}, a_{-6}, a_{-5}, a_{-4}, a_{-3}, a_{-2}, a_{-1}, a_{0} | a_{0} a_{-10} a_{-3}>


In [24]:
def get_two_letters(used_letters):
    available_letters = list(string.ascii_lowercase)
    greek_letters = ["alpha","beta","gamma","delta","epsilon","zeta","eta","theta","iota","kappa",
                     "lambda","mu","nu","xi","pi","rho","tau","phi","chi","psi","omega"]
    available_letters.extend(greek_letters)
    for letter in used_letters:
        available_letters.remove(letter)
    return available_letters[:2]

In [25]:
get_two_letters("abcdxyzf")

NameError: name 'string' is not defined

In [26]:
def magnus_case2(group):
    gen0 = generators[0]
    gen1 = generators[1]
        
    exp_sum0 = exponent_sumNEW(gen0,relation)
    exp_sum1 = exponent_sumNEW(gen1,relation)
        
    used_letters = {gen.ltr[:gen.index("_")] for gen in generators}
    letter0, letter1 = get_two_letters(used_letters)
    
    replacement0 = [Syllable(letter1, "", 1), Syllable(letter0, "", -exp_sum1)]
    replacement1 = [Syllable(letter0, "", exp_sum0)]
        
    for i in range(len(relation)):
        if relation[i] == gen0:
            relation = relation[:i] + replacement0 + relation[i+1:]
        elif relation[i] == gen1:
            relation = relation[:i] + replacement1 + relation[i+1:]
            
    # unfinished